# Modèle linéaire

Considérons la cas classique d'une fonction affine :

$$y=ax+b$$

Ici, $a$ et $b$ sont des réels. Ces deux nombres définissent entièrement la courbe et permet donc d'obtenir une relation **affine** entre $x$ et $y$. En statistique, cette relation est à la base des modèles dit **linéaires**, où une variable réponse se définit comme une somme de variables explicatives où chacune de ces dernières sont multipliés par un coefficient.


## Modèle linéaire simple

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/438px-Linear_regression.svg.png)

Dans le modèle linéaire simple (une seule variable explicative), on suppose que la variable réponse suit le modèle suivant :

$$y_i=\beta_0 + \beta_1 x_i + \varepsilon_i$$

On remarque la ressemblance avec la fonction affine présentée ci-dessus. La différence réside dans l'existence du terme aléatoire (appelé bruit) $\varepsilon_i$. Afin de considérer le modèle, il est nécessaire de se placer sous les hypothèses suivantes

$$(\mathcal{H}) : \left\{\begin{matrix}
\mathbb{E}[\varepsilon_i]=0\\ 
\text{Cov}(\varepsilon_i, \varepsilon_j)=\delta_{ij} \sigma^2
\end{matrix}\right.$$
Les différents éléments qui interviennent sont :

- $\beta_0$ : l'ordonnée à l'origine (nommée *intercept*)
- $\beta_1$ : le coefficient directeur
- $x_i$ : l'observation $i$
- $y_i$ : le $i$-ème prix
- $\varepsilon_i$ : le bruit aléatoire liée à la $i$-ème observation

La solution peut se calculer facilement via les formules fermées suivantes :

$$\hat{\beta}_1=\frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2} \qquad \hat{\beta}_0 = \hat{y} - \hat{\beta}_1 \bar{x}$$

## Modèle linéaire multiple

Dans le cas multiple (pour $p$ variables explicatives), pour la $i$-ème observation, le modèle s'écrit :

$$y_i= \beta_0 + \sum_{j=1}^p \beta_j x_{ij} + \varepsilon_i$$

Ainsi, une observation $x_i$ n'est plus une valeur, mais un **vecteur** $(x_{i1}, \dots, x_{ip})$. Il est plus commode de regrouper ces prix $y_i$ et ces vecteurs d'observations $x_i$ dans des matrices :

$$Y=X \beta + \varepsilon$$

Sous les hypothèses équivalentes du modèle simple en plus grand dimension

$$(\mathcal{H}) : \left\{\begin{matrix}
\text{rank}(X)=p\\ 
\mathbb{E}[\varepsilon]=0 \text{ et }\text{Var}(\varepsilon)=\sigma^2 I_p
\end{matrix}\right.$$

Les différents éléments qui interviennent sont :

- $\beta$ : le vecteur directeur
- $X$ : la matrice des observations
- $Y$ : le vecteur de prix
- $\varepsilon$ : le vecteur de bruit

Avec $X=( \mathbf{1}, X_1, \dots, X_n)$, $Y=(y_1, \dots, y_n)^\top$ et $\varepsilon=(\varepsilon_1, \dots, \varepsilon_n)^\top$. La solution des MCO (Moindres Carrés Ordinaires) est alors :

$$\hat{\beta}= (X^\top X)^{-1} X^\top Y$$

Vous pouvez d'ailleurs faire la démonstration de votre coté ! Pour plus d'information mathématiques, le portail de wikipédia qui est très bien fait : [lien ici](https://fr.wikipedia.org/wiki/Portail:Probabilit%C3%A9s_et_statistiques)

# 1) Implémenter une régression linéaire 


In [56]:
#importer vos librairies  
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn as sk;

In [42]:
#charger les données 
#price_availability.csv
#listings_final.csv
pa = pd.read_csv("price_availability.csv", sep=';')
lf = pd.read_csv("listings_final.csv", sep=';')

In [43]:
pa

,listing_id,day,created,available,local_currency,local_price,min_nights
0,9810829,2018-12-08,2018-09-27 06:14:10.000+0000,True,EUR,160,1
1,9810829,2018-12-08,2018-09-26 19:34:02.000+0000,True,EUR,160,1
2,20897010,2018-12-09,2018-09-27 10:38:57.000+0000,True,EUR,172,2
3,20897010,2018-12-09,2018-09-27 06:10:27.000+0000,True,EUR,172,2
4,20897010,2018-12-09,2018-09-26 19:30:25.000+0000,True,EUR,172,2
...,...,...,...,...,...,...,...
4748691,23608395,2018-09-06,2018-09-27 06:05:42.000+0000,False,EUR,24,1
4748692,23608395,2018-09-06,2018-09-26 19:31:32.000+0000,False,EUR,24,1
4748693,1447132,2018-12-27,2018-09-27 10:46:16.000+0000,False,EUR,125,3
4748694,1447132,2018-12-27,2018-09-27 06:07:36.000+0000,False,EUR,125,3


In [44]:
#lf.drop(['Unnamed: 0'], axis='columns', inplace = True)
lf.head()

,listing_id,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor
0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,1,2.0,False,True,True,False,False,1.00,1.00
1,661961,studio PARIS PLACE EDITH PIAF 75020,entire_home,Paris,NaN,48.867284,2.403255,2,1,1,1.0,False,False,True,True,False,0.88,0.69
2,1261705,chambre privée à louer @ paris oberkampf,private_room,Paris,NaN,48.867894,2.375897,1,1,1,1.0,False,False,True,True,False,1.00,1.00
3,1318834,Appartement au coeur du Marais,entire_home,Paris,République,48.870370,2.358510,3,2,2,1.0,False,False,True,False,False,0.82,0.48
4,1677091,Lovely & Quiet flat,entire_home,Paris,Buttes-Chaumont - Belleville,48.874149,2.373700,2,1,1,1.0,False,False,True,True,False,0.95,0.90


In [45]:
#vérifier si tous les individus ont bien un prix 
#pa['local_price'].isna().sum()
pa.isna().sum()

listing_id        0
day               0
created           0
available         0
local_currency    0
local_price       0
min_nights        0
dtype: int64

In [82]:
duplicas=pa.duplicated(keep=False,subset='listing_id')
doublons=pa[duplicas]

In [83]:
doublons

,listing_id,day,created,available,local_currency,local_price,min_nights
0,9810829,2018-12-08,2018-09-27 06:14:10.000+0000,True,EUR,160,1
1,9810829,2018-12-08,2018-09-26 19:34:02.000+0000,True,EUR,160,1
2,20897010,2018-12-09,2018-09-27 10:38:57.000+0000,True,EUR,172,2
3,20897010,2018-12-09,2018-09-27 06:10:27.000+0000,True,EUR,172,2
4,20897010,2018-12-09,2018-09-26 19:30:25.000+0000,True,EUR,172,2
...,...,...,...,...,...,...,...
4748691,23608395,2018-09-06,2018-09-27 06:05:42.000+0000,False,EUR,24,1
4748692,23608395,2018-09-06,2018-09-26 19:31:32.000+0000,False,EUR,24,1
4748693,1447132,2018-12-27,2018-09-27 10:46:16.000+0000,False,EUR,125,3
4748694,1447132,2018-12-27,2018-09-27 06:07:36.000+0000,False,EUR,125,3


In [85]:
grouped=doublons.groupby(['listing_id']).agg({'local_price':['mean']})
grouped=grouped.reset_index()

In [86]:
grouped

,listing_id,local_price
,,mean
0,5396,102.363985
1,7397,110.107632
2,9342,396.863014
3,10010,136.154856
4,10270,105.779221
...,...,...
11744,28836096,321.428571
11745,28838519,54.483333
11746,28840013,112.122530


In [102]:
grouped.columns = ['listing_id','local_price']
grouped

,listing_id,local_price
0,5396,102.363985
1,7397,110.107632
2,9342,396.863014
3,10010,136.154856
4,10270,105.779221
...,...,...
11744,28836096,321.428571
11745,28838519,54.483333
11746,28840013,112.122530
11747,28846494,168.037594


In [107]:
data_merge = inner_merged_total=pd.merge(grouped,lf,on=["listing_id"])

In [87]:
#pa['local_price'].groupby['listing_id']
#pa = pa.groupby(by="listing_id")
#pa.mean('local_price')

## 1.1) Données d'entrée

L'objectif ici est de charger les données pour créer les matrices $X$ et $Y$ du modèle linéaire. **Attention**, il n'est pas nécessaire de rajouter le vecteur colonne $\mathbf{1}$ en première colonne, car *scikit-learn* le fait automatiquement !

In [145]:
#définir 2 variables de travail
#X := les features à utiliser f
#Y := la target (prix)

X=data_merge[['listing_id','person_capacity','beds','bedrooms','bathrooms']]
Y=data_merge['local_price']

In [146]:
#construire l'ensemble de donnée prix 
#
#    INDICE 
# 
# récupérer les prix des ID dans le dataset de prix 
# 🚧 il y a plusieurs prix dans le dataset 🚧

pricing=pa.groupby('listing_id')['local_price'].agg([np.mean])
pricing.rename(columns={"mean": "mean_price"}, inplace=True)

pricing.head()

#pd.merge(pa['local_price'], on='listing_id')
#pd.merge(pa({'local_price'},{'local_currency'}), lf, on = ['listing_id'])

,mean_price
listing_id,
5396,102.363985
7397,110.107632
9342,396.863014
10010,136.154856
10270,105.779221


In [147]:
X.head()

,listing_id,person_capacity,beds,bedrooms,bathrooms
0,56093,4,2,1,1.0
1,57207,2,1,1,1.0
2,114543,2,1,1,1.0
3,149534,4,2,1,1.0
4,164255,4,2,1,1.0


In [148]:
Y.head()

0    170.000000
1     49.952756
2    107.374026
3    169.000000
4     75.876209
Name: local_price, dtype: float64

En *Machine Learning*, on a l'habitude de couper l'ensemble de données en deux sous-ensembles :

- Un ensemble d'entraînement (*train set*), sur lequel le modèle va être calibré.
- Un ensemble de test (*test set*), qui ne sera pas utilisé pendant le calibrage mais permettra de vérifier l'aptitude du modèle à généraliser sur de nouvelles observations inconnues.

En général, on découpe l'ensemble de données (*split*) en prenant $\alpha \%$ de l'ensemble pour entraînement et $1-\alpha \%$ comme test. Dans la plus part des cas, on considère que $\alpha=10,20 ou 30\%$.

In [149]:
#utiliser la méthode split de sklearn en splitant avec un alpha=30 et un random state=42 
#afficher la shape de vos données 

#pa.shape()

In [150]:
from sklearn.model_selection import train_test_split

In [151]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3,random_state=42)

In [152]:
X_train

,listing_id,person_capacity,beds,bedrooms,bathrooms
728,23348731,4,2,1,1.0
630,20926157,2,1,1,1.0
394,13489341,6,4,3,1.5
777,24515032,2,2,0,1.0
598,20011166,2,1,1,1.0
...,...,...,...,...,...
106,1410564,1,1,1,1.0
270,7039564,6,3,2,1.0
860,26271706,4,2,2,1.0
435,14956561,5,1,1,1.0


In [153]:
len(X_test)

300

In [159]:
from sklearn.linear_model import LinearRegression
lin_model = LinearRegression()

In [160]:
lin_model.fit(X_train, Y_train)

LinearRegression()

In [161]:
lin_model.predict(X_test)

array([ 133.96496952,  587.44247806,  104.2548419 ,  173.68751225,
        104.9639225 ,  349.55898985,  116.89955275, -290.48172103,
        122.34036368,  557.97046618,  220.66931361,  100.59121403,
        351.64703007,  175.88452004,  182.0221038 ,  220.17025059,
        217.14050493,  169.05705063,  328.68160205,  194.71831848,
        129.68181839,  118.57036255,  408.94757223,  142.42136512,
        142.58869552,  285.75658306,  146.63523211,   75.48826279,
        152.44948714,  116.32113516,   98.83597037,  123.27266995,
        145.10548476,  256.6221756 ,  100.23583135,  179.57028364,
         75.79944429,  271.83696984,  295.74335205,   98.89176428,
        139.02534009,  118.66348358,  234.36669024,  387.02501818,
         71.67401014,  129.97790304,  108.3914721 ,  136.95359615,
        133.82554892,  142.01907031,  223.07043815,  111.4025575 ,
        230.84800485,  230.48332611,  168.07461525,  317.78758809,
        132.34886416,  128.31713894,  471.80369039,  330.75843

In [162]:
Y_test

453      43.143689
793    1300.000000
209      57.886538
309     297.763158
740      55.349333
          ...     
314     195.000000
404     109.000000
7       300.000000
155      68.163588
809      73.782493
Name: local_price, Length: 300, dtype: float64

In [163]:
lin_model.score(X_test, Y_test)

0.42584752035528406

## 1.2) Entraînement

Pour information, *scikit-learn* utilise le solveur OLS (Ordinary Least Squares) de *numpy*.

In [164]:
#créer l'objet de régression et entrainer le sur notre ensemble d'entraînement

On affiche le vecteur des coefficients pour interpréter rapidement le modèle.

In [165]:
#afficher les coefficients
#que remarquez vous ? 

## 1.3) Validation du modèle

### Le coefficient de détermination $R^2$

Par la suite, nous ferons l'hypothèse de gaussianité sur les bruits. Dans l'idée, nous aimerions obtenir une valeur numérique qui nous indique à quel point la régression linéaire a un sens sur nos données. Pour cela, introduisons les notations suivantes :

- $SCT=\|Y-\hat{y} \mathbf{1}\|^2$ est la somme des carrés totaux
- $SCE=\|\hat{Y}-\hat{y} \mathbf{1}\|^2$ est la somme des carrés expliqués
- $SCR=\|\hat{\varepsilon}\|^2$ est la somme des carrés résiduels

L'idée est de décomposer la somme des carrés totaux comme la somme des carrés que le modèle explique, en plus de la somme des carrés qui sont liés aux résidus (et donc que le modèle ne peut pas expliquer). On voit donc ici l'intérêt de calculer un coefficient à partir du $SCE$. Puisque l'on a la relation suivante :

$$SCT=SCE+SCR \text{ alors } 1=\frac{SCE}{SCT}+\frac{SCR}{SCT}$$

Plus les résidus sont petits (et donc la régression est "bonne"), plus $SCR$ devient petit et donc $SCE$ devient grand. Le schéma inverse s'opère de la même façon. Dans le meilleur des cas, on obtient $SCR=0$ et donc $SCE=SCT$ d'où le premier membre vaut $1$. Dans le cas contraite, $SCE=0$ et automatiquement, le premier membre est nul. C'est ainsi que l'on définit le coefficient de détermination $R^2$ comme 
$$R^2=\frac{SCE}{SCT}=1-\frac{SCR}{SCT}$$
Ainsi, $R^2 \in [0,1]$. Plus $R^2$ est proche de $1$, plus la régression linéaire a du sens. Au contraire, si $R^2$ est proche de $0$, le modèle linéaire possède un faible pouvoir explicatif.

In [ ]:
#faire une prediction sur X


taille du vecteur Y_pred : 999 


In [ ]:
#afficher l'erreur des moindres carrées sur l'ensemble d'entrainement ainsi que le R2


Mean squared error: 21320.77
R2 : 0.37


## 1.4) Bonus : Analyse de l'homoscédasticité

L'analyse de l'homoscédasticité est primordiale : c'est en particulier elle qui nous permet de vérifier, à partir des résidus, si les bruits vérifient bien l'hypothèse $(\mathcal{H})$. On calcule donc les **résidus studentisés**.

$$t_i^*=\frac{\hat{\varepsilon}_i}{\hat{\sigma}_{(i)} \sqrt{1-h_{ii}}}$$
Avec $h_{ii}=\{X(X^\top X)^{-1} X^\top\}_{ii}=H_{ii}$ la matrice de projection sur l'hyperplan des variables. Plus précisément, $H$ est la matrice qui projette $Y$ sur l'espace engendré par les variables, soit $\hat{Y}=HY$. De même, on considère $\hat{\sigma}_{(i)}$ l'estimateur de la variance du bruit en supprimant l'observation $i$ (par une méthode de validation croisée Leave-One-Out que nous ne détaillerons pas ici).

Dans ce cas, on peut montrer que les résidus studentisés suivent une loi de Student à $n-p-1$ degrés de liberté.

In [55]:
#analyser le code ci-dessous 
import scipy
Y_pred = regr.predict(X_train)
n = X_train.shape[0]
p = 4
residuals = np.abs(y_train - Y_pred)
H = np.matmul(X_train, np.linalg.solve(np.dot(X_train.T, X_train), X_train.T))
std_hat = np.dot(residuals, residuals) / (n - p)
standart_residuals = np.asarray([residuals[i] / np.sqrt(std_hat * (1 - H[i, i])) for i in range(len(residuals))])
student_residuals = np.asarray([ standart_residuals[i] * np.sqrt((n - p - 1) / (n - p - standart_residuals[i]**2)) for i in range(n) ])
cook = np.asarray([ H[i, i] * student_residuals[i] / (X_train.shape[1] * (1 - H[i, i])) for i in range(n) ])

plt.figure(figsize=(20, 12))
plt.subplot(221)
plt.scatter(Y_pred, student_residuals, s=12, c="white", edgecolors="blue")
plt.plot([min(Y_pred), max(Y_pred)], [ scipy.stats.t.ppf(q=0.975, df=n-p-1), scipy.stats.t.ppf(q=0.975, df=n-p-1)], color="green", alpha=0.6, label="Quantile de Student")
plt.title("Analyse de l’homoscédasticité")
plt.xlabel("Prédictions $\hat{y}_i$")
plt.ylabel("Résidus studentisés $|t_i^*|$")
plt.legend()

NameError: name 'regr' is not defined